In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, roc_auc_score
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy, AUC, Accuracy
from tensorflow.keras.losses import BinaryCrossentropy, categorical_crossentropy, sparse_categorical_crossentropy, poisson
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

import xgboost as xgb
import lightgbm as lgb

from google.colab import drive
drive.mount('/drive', force_remount=True)
seed = 21
sns.set(style='white', context='notebook', palette='deep')

Mounted at /drive


In [ ]:
dataset = pd.read_csv("/drive/My Drive/dataset/dataset.csv", low_memory=False)
test = pd.read_csv("/drive/My Drive/dataset/test.csv", low_memory=False)
print(dataset.shape,test.shape)
dataset.head()

(20958, 8) (4261, 6)


,id,project_a,project_b,weight_a,weight_b,total_amount_usd,funder,quarter
0,715,https://github.com/mochajs/mocha,https://github.com/chzyer/readline,0.961581,0.038419,2681,opencollective,2016-04
1,716,https://github.com/mochajs/mocha,https://github.com/gulpjs/gulp,0.992302,0.007698,2598,opencollective,2016-04
2,717,https://github.com/chzyer/readline,https://github.com/gulpjs/gulp,0.837398,0.162602,123,opencollective,2016-04
3,718,https://github.com/chzyer/readline,https://github.com/gulpjs/gulp,0.231595,0.768405,652,opencollective,2016-07
4,719,https://github.com/chzyer/readline,https://github.com/mochajs/mocha,0.078851,0.921149,1915,opencollective,2016-07


In [ ]:
test.head()

,id,project_a,project_b,total_amount_usd,funder,quarter
0,20884,https://github.com/rollup/rollup,https://github.com/webpack/webpack,29097,opencollective,2024-01
1,20885,https://github.com/rollup/rollup,https://github.com/babel/babel,26093,opencollective,2024-01
2,20886,https://github.com/rollup/rollup,https://github.com/sindresorhus/type-fest,4460,opencollective,2024-01
3,20887,https://github.com/rollup/rollup,https://github.com/inikulin/parse5,2454,opencollective,2024-01
4,20888,https://github.com/rollup/rollup,https://github.com/zloirock/core-js,13229,opencollective,2024-01


In [ ]:
import pandas as pd
import requests
!pip install PyGithub
from github import Github

# Function to fetch GitHub stars
def get_github_stars(owner, repo):
    url = f"https://api.github.com/repos/{owner}/{repo}"
    response = requests.get(url)
    data = response.json()
    return data.get("stargazers_count", 0)

# Function to extract owner and repo from URL
def extract_owner_repo(url):
    parts = url.split('/')
    return parts[-2], parts[-1]

# Initialize new columns
dataset['stars_a'] = 0
dataset['stars_b'] = 0

# Fetch stars and comments
for index, row in dataset.iterrows():
    owner_a, repo_a = extract_owner_repo(row['project_a'])
    owner_b, repo_b = extract_owner_repo(row['project_b'])

    dataset.at[index, 'stars_a'] = get_github_stars(owner_a, repo_a)
    dataset.at[index, 'stars_b'] = get_github_stars(owner_b, repo_b)

In [ ]:
# Data Preparation
# Drop unnecessary columns and handle any missing values if necessary
# Data Preparation
# Drop unnecessary columns and handle any missing values if necessary
X = dataset.drop(['id', 'weight_a', 'weight_b', 'funder', 'quarter'], axis=1)
y = dataset['weight_a']  # Target variable
# Include only GitHub stars
X = X[['project_a', 'project_b', 'total_amount_usd', 'stars_a', 'stars_b']]



# Now perform LabelEncoding on 'project_a', 'project_b' in the original dataset
categorical_features = ['project_a', 'project_b']
for col in categorical_features:
    le = LabelEncoder()
    # Fit on combined data to ensure consistent encoding
    le.fit(pd.concat([dataset[col], test[col]]))
    X[col] = le.transform(dataset[col])  # Encode and assign to X
    test[col] = le.transform(test[col]) # Encode and assign to test

# Now do the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create LightGBM datasets
# (Note: categorical_feature is removed as columns are already numerical)
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Set parameters for LightGBM
params = {
    'objective': 'regression',
    'metric': 'mse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.02,  # Reduced learning rate
    'num_leaves': 31,       # Adjusted num_leaves
    'max_depth': 5,         # Limit tree depth
    'min_data_in_leaf': 150, # Increase min_data_in_leaf
    'feature_fraction': 1, # Use a subset of features
    'bagging_fraction': 1, # Bagging
    'lambda_l1': 0.01,     # L1 regularization
    'lambda_l2': 0.01,     # L2 regularization
    'verbose': -5
}

# Train the model
model = lgb.train(params,
                  train_data,
                  valid_sets=[test_data],
                  num_boost_round=20000,  # Increased boosting rounds
                  callbacks=[lgb.early_stopping(stopping_rounds=1000)])


# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# ... (Model training and evaluation) ...

# Calculate 'stars_a' and 'stars_b' for the test DataFrame
# Initialize new columns
# Create a temporary DataFrame to store original 'project_a', 'project_b', 'stars_a', 'stars_b' for test data
temp_df_test = pd.DataFrame(index=test.index)  # Create empty DataFrame with same index
temp_df_test['stars_a'] = 0
temp_df_test['stars_b'] = 0

# **Create a copy of the test DataFrame with original URL values**
test_original_urls = dataset[['project_a', 'project_b']][dataset.index.isin(test.index)].copy()

# Fetch stars and comments using original 'project_a', 'project_b' values from test_original_urls
for index, row in test_original_urls.iterrows():  # Iterating through test_original_urls DataFrame
    owner_a, repo_a = extract_owner_repo(row['project_a'])  # Using original 'project_a'
    owner_b, repo_b = extract_owner_repo(row['project_b'])  # Using original 'project_b'

    temp_df_test.at[index, 'stars_a'] = get_github_stars(owner_a, repo_a)
    temp_df_test.at[index, 'stars_b'] = get_github_stars(owner_b, repo_b)

# Merge the 'stars_a', 'stars_b' columns back into the test DataFrame
test = test.merge(temp_df_test[['stars_a', 'stars_b']], left_index=True, right_index=True)

# ... (Rest of your code) ...



# Prepare the submission file
# Drop unnecessary columns, including 'total_amount_usd'
# Create a list of columns to keep for prediction, these must match the columns in X_train
features_to_keep = ['project_a', 'project_b', 'total_amount_usd', 'stars_a','stars_b']
test['weight_a_pred'] = model.predict(test[features_to_keep])


# ... (Rest of the code) ...


# Prepare the submission file
submission = test[['id']].copy()
submission['pred'] = test['weight_a_pred']
submission.to_csv('/drive/My Drive/dataset/2stsubmission.csv', index=False)

print(" First Submission file created successfully.")

# Prepare the submission file
# Ensure transitivity in predictions
project_ranks = {}
predictions = test[['project_a', 'project_b', 'weight_a_pred']].copy()
predictions.columns = ['project_a', 'project_b', 'predicted_weight_a']

# Assign initial ranks based on predicted weights
for index, row in predictions.iterrows():
    project_a = row['project_a']
    project_b = row['project_b']
    weight_a = row['predicted_weight_a']

    if project_a not in project_ranks:
        project_ranks[project_a] = 0
    if project_b not in project_ranks:
        project_ranks[project_b] = 0

    if weight_a > 0.5:
        project_ranks[project_a] += 1
        project_ranks[project_b] -= 1
    else:
        project_ranks[project_a] -= 1
        project_ranks[project_b] += 1

# Adjust predictions based on project rankings
for index, row in predictions.iterrows():
    project_a = row['project_a']
    project_b = row['project_b']

    if project_ranks[project_a] > project_ranks[project_b]:
        predictions.loc[index, 'predicted_weight_a'] = 1.0  # A > B
    elif project_ranks[project_a] < project_ranks[project_b]:
        predictions.loc[index, 'predicted_weight_a'] = 0.0  # A < B
    else:
        predictions.loc[index, 'predicted_weight_a'] = 0.5  # A = B

# Normalize the weights to ensure they sum to 1 for each pair
# Assuming you have a DataFrame `predictions` with columns ['project_a', 'project_b', 'predicted_weight_a']
predictions['predicted_weight_b'] = 1 - predictions['predicted_weight_a']

# Ensure that the weights are between 0 and 1
predictions['predicted_weight_a'] = predictions['predicted_weight_a'].clip(0, 1)
predictions['predicted_weight_b'] = predictions['predicted_weight_b'].clip(0, 1)

# Prepare the submission file
submission = test[['id']].copy()
submission['pred'] = predictions['predicted_weight_a'].values  # Use the adjusted weights

# Save the submission file
submission.to_csv('/drive/My Drive/dataset/NEW1-IMPROVED-submission.csv', index=False)

print("Submission file created successfully with transitivity enforced.")



Training until validation scores don't improve for 1000 rounds
Did not meet early stopping. Best iteration is:
[19987]	valid_0's l2: 0.0464557
Mean Squared Error: 0.04645565892943879
 First Submission file created successfully.
Submission file created successfully with transitivity enforced.
